In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.preprocessing import LabelEncoder

In [2]:
mental_df = pd.read_csv("Resources/Indicators_of_Anxiety_or_Depression_Based_on_Reported_Frequency_of_Symptoms_During_Last_7_Days.csv")
mental_df.head()

,Indicator,Group,State,Subgroup,Phase,Time Period,Time Period Label,Time Period Start Date,Time Period End Date,Value,Low CI,High CI,Confidence Interval,Quartile Range
0,Symptoms of Depressive Disorder,National Estimate,United States,United States,1,1,"Apr 23 - May 5, 2020",04/23/2020,05/05/2020,23.5,22.7,24.3,22.7 - 24.3,NaN
1,Symptoms of Depressive Disorder,By Age,United States,18 - 29 years,1,1,"Apr 23 - May 5, 2020",04/23/2020,05/05/2020,32.7,30.2,35.2,30.2 - 35.2,NaN
2,Symptoms of Depressive Disorder,By Age,United States,30 - 39 years,1,1,"Apr 23 - May 5, 2020",04/23/2020,05/05/2020,25.7,24.1,27.3,24.1 - 27.3,NaN
3,Symptoms of Depressive Disorder,By Age,United States,40 - 49 years,1,1,"Apr 23 - May 5, 2020",04/23/2020,05/05/2020,24.8,23.3,26.2,23.3 - 26.2,NaN
4,Symptoms of Depressive Disorder,By Age,United States,50 - 59 years,1,1,"Apr 23 - May 5, 2020",04/23/2020,05/05/2020,23.2,21.5,25.0,21.5 - 25.0,NaN


In [3]:
mental_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8901 entries, 0 to 8900
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Indicator               8901 non-null   object 
 1   Group                   8901 non-null   object 
 2   State                   8901 non-null   object 
 3   Subgroup                8901 non-null   object 
 4   Phase                   8901 non-null   object 
 5   Time Period             8901 non-null   int64  
 6   Time Period Label       8901 non-null   object 
 7   Time Period Start Date  8901 non-null   object 
 8   Time Period End Date    8901 non-null   object 
 9   Value                   8604 non-null   float64
 10  Low CI                  8604 non-null   float64
 11  High CI                 8604 non-null   float64
 12  Confidence Interval     8604 non-null   object 
 13  Quartile Range          6120 non-null   object 
dtypes: float64(3), int64(1), object(10)
memo

In [4]:
#Clean out unused columns
mental_df=mental_df.drop(columns=["Low CI", "High CI", "Confidence Interval", "Quartile Range", "Phase", "Time Period", "Time Period End Date", "Time Period Label","State"])

In [5]:
mental_df['Value'].describe()

count    8604.000000
mean       30.146897
std         7.763409
min         6.400000
25%        24.700000
50%        29.700000
75%        35.100000
max        71.900000
Name: Value, dtype: float64

In [6]:
#Change Values into risk categories for ML
mental_df["Risk"] = ""
mental_df.loc[(mental_df["Value"] <=24 ), "Risk"] = "Mild"
mental_df.loc[(mental_df["Value"] >24 ) & (mental_df['Value'] <=30 ), "Risk"] = "Moderate"
mental_df.loc[(mental_df["Value"] >30 ) & (mental_df['Value'] <=35 ), "Risk"] = "High"
mental_df.loc[(mental_df["Value"] >35 ), "Risk"] = "Extreme"

In [7]:
mental_df.to_csv("Resources/Almost_Clean.csv")

In [8]:
mental_df.head()

,Indicator,Group,Subgroup,Time Period Start Date,Value,Risk
0,Symptoms of Depressive Disorder,National Estimate,United States,04/23/2020,23.5,Mild
1,Symptoms of Depressive Disorder,By Age,18 - 29 years,04/23/2020,32.7,High
2,Symptoms of Depressive Disorder,By Age,30 - 39 years,04/23/2020,25.7,Moderate
3,Symptoms of Depressive Disorder,By Age,40 - 49 years,04/23/2020,24.8,Moderate
4,Symptoms of Depressive Disorder,By Age,50 - 59 years,04/23/2020,23.2,Mild


In [9]:
#Create year column for further table manipulation
mental_df['Time Period Start Date'] = pd.to_datetime(mental_df['Time Period Start Date'])
mental_df['Time Period Start Date'] = pd.to_datetime(mental_df['Time Period Start Date'],format='%Y%m%d')
mental_df['year'] = pd.DatetimeIndex(mental_df['Time Period Start Date']).year
mental_df = mental_df.rename(columns={"Time Period Start Date": "Time_Period"})
mental_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8901 entries, 0 to 8900
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Indicator    8901 non-null   object        
 1   Group        8901 non-null   object        
 2   Subgroup     8901 non-null   object        
 3   Time_Period  8901 non-null   datetime64[ns]
 4   Value        8604 non-null   float64       
 5   Risk         8901 non-null   object        
 6   year         8901 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 486.9+ KB


In [10]:
mental_df['Group'].unique()

array(['National Estimate', 'By Age', 'By Sex',
       'By Race/Hispanic ethnicity', 'By Education', 'By State',
       'By Disability status', 'By Gender identity',
       'By Sexual orientation'], dtype=object)

In [11]:

#We could change the group column into seperate columns, and use subgroup as the data.
#example: Column:By age, not sorted by age = 0, 18-29 = 1, 30-39 = 2
#Would reduce from 91 columns down to 10? ish columns
#91 columns still works properly though if it's too much work



#Needs to be worked on, currently creates 187 columns lol


























#Removed "State" column as it was redundant, down to 135 columns



#Imported sklearn label encoder, changed time period column, now down to 91 columns

# -------------------- code is below ------------------------------------------------


#le = LabelEncoder()
#mental_df['Time_Period'] = le.fit_transform(mental_df['Time_Period'])

#mental_df = pd.get_dummies(mental_df, columns=["Group","Subgroup"])

In [12]:
mental_2020_df = mental_df.loc[(mental_df['year'] == 2020)]
mental_2021_df = mental_df.loc[(mental_df['year'] == 2021)]
del mental_2020_df['year']
del mental_2021_df['year']


In [13]:
mental_2020_df.head()

,Indicator,Group,Subgroup,Time_Period,Value,Risk
0,Symptoms of Depressive Disorder,National Estimate,United States,2020-04-23,23.5,Mild
1,Symptoms of Depressive Disorder,By Age,18 - 29 years,2020-04-23,32.7,High
2,Symptoms of Depressive Disorder,By Age,30 - 39 years,2020-04-23,25.7,Moderate
3,Symptoms of Depressive Disorder,By Age,40 - 49 years,2020-04-23,24.8,Moderate
4,Symptoms of Depressive Disorder,By Age,50 - 59 years,2020-04-23,23.2,Mild


In [14]:
mental_2021_df.head()

,Indicator,Group,Subgroup,Time_Period,Value,Risk
4486,Symptoms of Depressive Disorder,National Estimate,United States,2021-01-06,28.4,Moderate
4487,Symptoms of Depressive Disorder,By Age,18 - 29 years,2021-01-06,41.1,Extreme
4488,Symptoms of Depressive Disorder,By Age,30 - 39 years,2021-01-06,31.9,High
4489,Symptoms of Depressive Disorder,By Age,40 - 49 years,2021-01-06,29.1,Moderate
4490,Symptoms of Depressive Disorder,By Age,50 - 59 years,2021-01-06,27.1,Moderate


In [15]:
mental_2020_df.to_csv("Resources/Almost_Clean_2020.csv")
mental_2021_df.to_csv("Resources/Almost_Clean_2021.csv")
del mental_2020_df['Value']
del mental_2021_df['Value']

In [16]:
#Depression 2020
mental_2020_Depression_df=mental_2020_df.loc[(mental_df['Indicator'] == 'Symptoms of Depressive Disorder')]
del mental_2020_Depression_df['Indicator']
mental_2020_Depression_df.head()

,Group,Subgroup,Time_Period,Risk
0,National Estimate,United States,2020-04-23,Mild
1,By Age,18 - 29 years,2020-04-23,High
2,By Age,30 - 39 years,2020-04-23,Moderate
3,By Age,40 - 49 years,2020-04-23,Moderate
4,By Age,50 - 59 years,2020-04-23,Mild


In [17]:
#Anxiety 2020
mental_2020_Anxiety_df=mental_2020_df.loc[(mental_df['Indicator'] == 'Symptoms of Anxiety Disorder')]
del mental_2020_Anxiety_df['Indicator']
mental_2020_Anxiety_df.head()

,Group,Subgroup,Time_Period,Risk
70,National Estimate,United States,2020-04-23,High
71,By Age,18 - 29 years,2020-04-23,Extreme
72,By Age,30 - 39 years,2020-04-23,High
73,By Age,40 - 49 years,2020-04-23,High
74,By Age,50 - 59 years,2020-04-23,High


In [18]:
#Either 2020
mental_2020_AnxOrDPRS_df=mental_2020_df.loc[(mental_df['Indicator'] == 'Symptoms of Anxiety Disorder or Depressive Disorder')]
del mental_2020_AnxOrDPRS_df['Indicator']
mental_2020_AnxOrDPRS_df.head()

,Group,Subgroup,Time_Period,Risk
140,National Estimate,United States,2020-04-23,Extreme
141,By Age,18 - 29 years,2020-04-23,Extreme
142,By Age,30 - 39 years,2020-04-23,Extreme
143,By Age,40 - 49 years,2020-04-23,Extreme
144,By Age,50 - 59 years,2020-04-23,Extreme


In [19]:
#Depression 2021
mental_2021_Depression_df=mental_2021_df.loc[(mental_df['Indicator'] == 'Symptoms of Depressive Disorder')]
del mental_2021_Depression_df['Indicator']
mental_2021_Depression_df.head()

,Group,Subgroup,Time_Period,Risk
4486,National Estimate,United States,2021-01-06,Moderate
4487,By Age,18 - 29 years,2021-01-06,Extreme
4488,By Age,30 - 39 years,2021-01-06,High
4489,By Age,40 - 49 years,2021-01-06,Moderate
4490,By Age,50 - 59 years,2021-01-06,Moderate


In [20]:
#Anxiety 2021
mental_2021_Anxiety_df=mental_2021_df.loc[(mental_df['Indicator'] == 'Symptoms of Anxiety Disorder')]
del mental_2021_Anxiety_df['Indicator']
mental_2021_Anxiety_df.head()

,Group,Subgroup,Time_Period,Risk
4575,National Estimate,United States,2021-01-06,Extreme
4576,By Age,18 - 29 years,2021-01-06,Extreme
4577,By Age,30 - 39 years,2021-01-06,Extreme
4578,By Age,40 - 49 years,2021-01-06,Extreme
4579,By Age,50 - 59 years,2021-01-06,Extreme


In [21]:
#Either 2021
mental_2021_AnxOrDPRS_df=mental_2021_df.loc[(mental_df['Indicator'] == 'Symptoms of Anxiety Disorder or Depressive Disorder')]
del mental_2021_AnxOrDPRS_df['Indicator']
mental_2021_AnxOrDPRS_df.head()

,Group,Subgroup,Time_Period,Risk
4664,National Estimate,United States,2021-01-06,Extreme
4665,By Age,18 - 29 years,2021-01-06,Extreme
4666,By Age,30 - 39 years,2021-01-06,Extreme
4667,By Age,40 - 49 years,2021-01-06,Extreme
4668,By Age,50 - 59 years,2021-01-06,Extreme
